## Spring 4 (nivel I)

Primero cargaremos los datos del log y echaremos un vistazo a los datos:


In [8]:

# leamos el archivo
data_txt=open("Web_access_log-akumenius.com.txt","r")

#vemos las primeras lineas
data_txt.readline()

'localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-\n'

Vemos un poco más del archivo

In [9]:
data_txt.readlines(1000)
# parece demasíado fácil , amplíamos un poco más la muestra

['localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-\n',
 'localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-\n',
 'localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-\n',
 'localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-\n',
 'localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-\n',
 'localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-\n',
 'localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-\n',
 'localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTION

In [10]:
data_txt.readlines(5000)

['localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-\n',
 'localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-\n',
 'localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-\n',
 'localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-\n',
 'localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-\n',
 'localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-\n',
 'localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-\n',
 'localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTION

Vemos que no tiene el formato csv estandard .Si intentamos cargar el archivo usando el espacio como separador.

import pandas as pd
logs_csv=pd.read_table("Web_access_log-akumenius.com.txt",delimiter=" ",header=0)

obtendremos un error :

---------------------------------------------------------------------------
ParserError                               Traceback (most recent call last)

....
....

c:\Users\anton\anaconda3\lib\site-packages\pandas\_libs\parsers.pyx in pandas._libs.parsers.TextReader._tokenize_rows()

c:\Users\anton\anaconda3\lib\site-packages\pandas\_libs\parsers.pyx in pandas._libs.parsers.raise_parser_error()

ParserError: Error tokenizing data. C error: Expected 12 fields in line 181474, saw 13

Los datos no són suficientemente homogeneos para hacerlo de esta manera. Para ello usaremos las regular expresions.
Pero antes de ello debemos entender la estructura básica del log. Buscando un poco podemos descubrir que es un archivo log de apache y definir cada uno de los campos de los que esta compuesto.(fuente : https://www.sumologic.com/blog/apache-access-log/).

Analizamos una de las lineas anteriores para identificar los campos:


'<span style="color:red">www.akumenius.com</span><span style="color:blue"> 66.249.76.216</span> - - <span style="color:yellow">[23/Feb/2014:03:10:53 +0100]</span><span style="color:green"><br> "GET /destinos-baratos/destinosEstrelles/hoteles-en-Villarcayo_ESPA%C3%91A-con-1-estrella HTTP/1.1"</span><span style="color:yellow"> 200</span><span style="color:blue"> 8810</span> "-"<span style="color:red"> "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" VLOG=</span><br>-\n']



WEB:<span style="color:red"> www.akumenius.com</span><br>
IP: <span style="color:blue">66.249.76.216</span><br>
Fecha: <span style="color:yellow">23/Feb/2014</span><br>
Hora: <span style="color:orange">03:10:31 +0100</span><br>
Tipo de acceso y consulta: <span style="color:green">"GET /hoteles-baratos/ofertas-hotel-Club-&-Hotel-Letoonia--en-Fethiye-8460b-destinos.html HTTP/1.1"</span><br>
HTTP response status code: <span style="color:yellow">404</span><br>
Bytes transferidos: <span style="color:blue">3100</span><br>
user agent:<span style="color:red">"Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"</span><br>


Ahora que ya tenemos una estructura , vamos montando las regEX de cada parte

En realidad se podría hacer una regEx única , pero si hay cualquier formato que difiera en alguna parte de la expresión , falla todo la identificación , así que lo debemos hacer es ir buscando los grupos por separado y controlando si hay coincidencia o no.

_En este ejercicio,como demostración académica , en el algoritmo se introduce las 2 maneras para mostrar su funcionamiento._<br>
_Es decir , se usará un <span style="color:green">**if**</span> parar usar la expresión completa y en el <span style="color:green">**else:**</span> se hará las búsqueda grupo por grupo._

(*)_Antes samplearemos el archivo de trabajo del log , para que pueda ser procesable en unos tiempos razonables_



In [11]:
import re
import pandas as pd

data_sample=data_txt.readlines(5000000)

Nos interesan los accesos que hacen desde internet, para ello definimos un filtro para no contar con los accesos desde el localhost.

In [12]:
def filtro_localhost(linea):
    localhost=True
    #buscamos localhost en cada linea , si lo encuentra devuelve false.
    if (re.findall(r"localhost",linea)):
        localhost=False
    return localhost

In [13]:


log_df=pd.DataFrame()
log_datos={}#crear dictcionarioo con campos

for line in data_sample:
    if(filtro_localhost(line)):
    #sera True si no es localhost.Ejecutamos la preparación de la cadena de texto
    #Primero partimos la parte de la web +IP+hora del resto y lo almacenamos en un array , para después pasarlo a un diccionario
     
        string_IP_Time=re.search(r"(\w{3}\.\w+\.\w{3})\D+(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}).+(\d{2,2}/\D{3,3}/\d{4,4}):(\d{2}:\d{2}:\d{2} \+\d{4}).+(\"([A-Z]{1,5}).(.+?)\").(\d{3}).(\d+).+(\"(.+?)\")",line)
        
        #mirar si cumple con la separacion de la linea
        if(string_IP_Time!=None):

            log_datos["WEB"]=string_IP_Time.group(1)
            log_datos["IP"]=string_IP_Time.group(2)
            log_datos["Fecha"]=string_IP_Time.group(3)
            log_datos["Hora"]=string_IP_Time.group(4)
            log_datos["Tipo acceso"]=string_IP_Time.group(6)
            log_datos["local acceso"]=string_IP_Time.group(7)
            log_datos["msg http"]=string_IP_Time.group(8)
            log_datos["bytes"]=string_IP_Time.group(9)
            log_datos["boot"]=string_IP_Time.group(11)
            #agregamos los datos al dataframe
            log_df=log_df.append(log_datos,ignore_index=True)
        else:
            #la misma separeación que antes pero ahora de forma separada
    
            web=re.search(r"(\w{3}\.\w+\.\w{3}).+",line)
            if(web!=None):
                log_datos["WEB"]=web.group(1)
            else:
                log_datos["WEB"]="NA"

            ip=re.search(r"(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}).+",line)
            if(ip!=None):
                log_datos["IP"]=ip.group(1)
            else: 
                log_datos["IP"]="NA"   

            date=re.search(r"(\d{2,2}/\D{3,3}/\d{4,4}):.+",line)
            if(date!=None):
                log_datos["Fecha"]=date.group(1)
            else:
                log_datos["Fecha"]="NA"

            time=re.search(r"(\d{2}:\d{2}:\d{2} \+\d{4}).+",line)
            if(time!=None):
                log_datos["Hora"]=time.group(1)
            else:
                log_datos["Hora"]="NA"
            url=re.search(r"(\"([A-Z]{1,5}).(.+?)\").+",line)
            if(url!=None):

                log_datos["Tipo acceso"]=url.group(2)
                log_datos["local acceso"]=url.group(3)
            else:
                log_datos["Tipo acceso"]="NA"
                log_datos["local acceso"]="NA"

            htp_code=re.search(r".+\".(\d{3}).(\d+).+",line)
            if(htp_code!=None):

                log_datos["msg http"]=htp_code.group(1)
                log_datos["bytes"]=htp_code.group(2)
            else:
                log_datos["msg http"]="NA"
                log_datos["bytes"]="NA"

            bot=re.search(r"(\"(.+?)\").+(\"(.+?)\")",line)
            if (bot!=None):
                log_datos["boot"]=bot.group(4)
            else:
                log_datos["boot"]="NA"
        
        log_df=log_df.append(log_datos,ignore_index=True)



Vemos la info del dataframe obtenido.

In [14]:
log_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36196 entries, 0 to 36195
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   WEB           36196 non-null  object
 1   IP            36196 non-null  object
 2   Fecha         36196 non-null  object
 3   Hora          36196 non-null  object
 4   Tipo acceso   36196 non-null  object
 5   local acceso  36196 non-null  object
 6   msg http      36196 non-null  object
 7   bytes         36196 non-null  object
 8   boot          36196 non-null  object
dtypes: object(9)
memory usage: 2.5+ MB


In [15]:
log_df.describe()

,WEB,IP,Fecha,Hora,Tipo acceso,local acceso,msg http,bytes,boot
count,36196,36196,36196,36196,36196,36196,36196,36196,36196
unique,9,342,1,14159,3,14220,5,4474,65
top,www.akumenius.com,66.249.76.216,23/Feb/2014,11:21:58 +0100,GET,/ HTTP/1.1,200,3100,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...
freq,36150,18702,36196,56,35774,282,29526,6514,18562


In [16]:
log_df.head()


,WEB,IP,Fecha,Hora,Tipo acceso,local acceso,msg http,bytes,boot
0,www.akumenius.com,66.249.76.216,23/Feb/2014,03:10:54 +0100,GET,/hoteles-baratos/ofertas-hotel-Hotel-Costa-del...,404,3100,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...
1,www.akumenius.com,66.249.76.216,23/Feb/2014,03:10:54 +0100,GET,/hoteles-baratos/ofertas-hotel-Hotel-Costa-del...,404,3100,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...
2,www.akumenius.com,5.255.253.53,23/Feb/2014,03:10:56 +0100,GET,/destinos-baratos/hoteles-baratos/ofertas-hote...,404,3100,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...
3,www.akumenius.com,5.255.253.53,23/Feb/2014,03:10:56 +0100,GET,/destinos-baratos/hoteles-baratos/ofertas-hote...,404,3100,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...
4,www.akumenius.com,66.249.76.216,23/Feb/2014,03:10:56 +0100,GET,/hoteles-baratos/ofertas-hotel-Quintessia-&-Sp...,404,3100,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...


Analizamos el campo _WEB_ por separado.

In [17]:
web=log_df["WEB"]


In [18]:
web.describe()


count                 36196
unique                    9
top       www.akumenius.com
freq                  36150
Name: WEB, dtype: object

In [19]:
web.unique()

array(['www.akumenius.com', '220.181.108', 'www.bing.com', 'NA',
       '119.63.193', 'www.google.com', 'www.baidu.com', '218.30.103',
       'rch.msn.com'], dtype=object)

Vemos que hay algunas lineas que estan mal separadas , ya que tenemos datos IP en el campo _WEB_ .
Vamos a filtrar quitando los valores que no coinciden con lo esperado en el campo _WEB_

In [20]:
for x in log_df.index:
    if (log_df.loc[x,"WEB"]!='www.akumenius.com'):
        log_df.drop(x,inplace=True)

In [21]:
log_df.describe()

,WEB,IP,Fecha,Hora,Tipo acceso,local acceso,msg http,bytes,boot
count,36150,36150,36150,36150,36150,36150,36150,36150,36150
unique,1,330,1,14132,3,14211,4,4471,65
top,www.akumenius.com,66.249.76.216,23/Feb/2014,11:21:58 +0100,GET,/ HTTP/1.1,200,3100,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...
freq,36150,18702,36150,56,35728,261,29526,6514,18558


Vemos que hay un 10% aprox de líneas mal separadas. Para continuar usaré un archivo csv obtenido de aplicar las regular expresions al archivo log original. 

In [22]:

df=pd.read_csv("log_exp.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116360 entries, 0 to 116359
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Unnamed: 0    116360 non-null  int64 
 1   WEB           116360 non-null  object
 2   IP            116360 non-null  object
 3   Fecha         116360 non-null  object
 4   Hora          116360 non-null  object
 5   Tipo acceso   116360 non-null  object
 6   local acceso  116360 non-null  object
 7   msg http      116360 non-null  int64 
 8   bytes         116360 non-null  int64 
 9   boot          116360 non-null  object
dtypes: int64(3), object(7)
memory usage: 8.9+ MB


Vemos que partimos de una muestra de 116360 
Procedemos como con el dataframe sample anterior

In [23]:
df["WEB"].unique()

array(['www.akumenius.com', 'www.bing.com', 'www.google.com',
       '249.76.191', 'www.baidu.com', '193.116.190', 'est.akumenius.com',
       '220.181.108', '119.63.193', '157.56.229', '125.71.222',
       '195.128.174', '108.168.250', '213.251.189', 'rch.msn.com',
       '144.76.183', '193.43.252', '187.48.112', '193.120.106',
       '193.110.219', '188.165.195', '109.205.137', '188.40.249',
       '178.110.118', '237.100.185', '165.21.153', '226.40.145',
       '208.88.225', '103.65.162', 'com.apple.Map'], dtype=object)

In [24]:
for x in df.index:
    if (df.loc[x,"WEB"]!='www.akumenius.com'):
        df.drop(x,inplace=True)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112556 entries, 0 to 116359
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Unnamed: 0    112556 non-null  int64 
 1   WEB           112556 non-null  object
 2   IP            112556 non-null  object
 3   Fecha         112556 non-null  object
 4   Hora          112556 non-null  object
 5   Tipo acceso   112556 non-null  object
 6   local acceso  112556 non-null  object
 7   msg http      112556 non-null  int64 
 8   bytes         112556 non-null  int64 
 9   boot          112556 non-null  object
dtypes: int64(3), object(7)
memory usage: 13.5+ MB


Vemos que con la muestra ampliada vemos que el ratio de error es más bajo que en el sample que haíamos cogido antes. Por lo tanto , a primera vista podemos dar por bueno las reg Ex para poder normalizar el archivo log.
Ahora podríamos hacer un sample para trabajar y normalizar los datos .Por ejemplo en el tipo de acceso podemos ver los diferentes tipos 

In [26]:
df["Tipo acceso"].unique()

array(['GET', 'POST', 'HEAD'], dtype=object)

A partir de aqui podríamos procesar los datos según nuestro ínteres en los datos , tal como hemos hecho en ejercicios anteriores.